In [ ]:
import argparse
import math
import os
from copy import deepcopy
from datetime import datetime
from pathlib import Path
from typing import List, Union

import numpy as np
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from ultralytics import YOLO, __version__
from ultralytics.nn.modules import Detect, C2f, Conv, Bottleneck
from ultralytics.nn.tasks import attempt_load_one_weight
from ultralytics.yolo.engine.model import TASK_MAP
from ultralytics.yolo.engine.trainer import BaseTrainer
from ultralytics.yolo.utils import yaml_load, LOGGER, RANK, DEFAULT_CFG_DICT, DEFAULT_CFG_KEYS
from ultralytics.yolo.utils.checks import check_yaml
from ultralytics.yolo.utils.torch_utils import initialize_weights, de_parallel

import torch_pruning as tp
from fasterai.prune.all import *
from fastai.vision.all import *
from fastcore.basics import store_attr, listify, true
from torch_pruning.pruner import function

## Helpers

In [ ]:
def infer_shortcut(bottleneck):
    c1 = bottleneck.cv1.conv.in_channels
    c2 = bottleneck.cv2.conv.out_channels
    return c1 == c2 and hasattr(bottleneck, 'add') and bottleneck.add


class C2f_v2(nn.Module):
    # CSP Bottleneck with 2 convolutions
    def __init__(self, c1, c2, n=1, shortcut=False, g=1, e=0.5):  # ch_in, ch_out, number, shortcut, groups, expansion
        super().__init__()
        self.c = int(c2 * e)  # hidden channels
        self.cv0 = Conv(c1, self.c, 1, 1)
        self.cv1 = Conv(c1, self.c, 1, 1)
        self.cv2 = Conv((2 + n) * self.c, c2, 1)  # optional act=FReLU(c2)
        self.m = nn.ModuleList(Bottleneck(self.c, self.c, shortcut, g, k=((3, 3), (3, 3)), e=1.0) for _ in range(n))

    def forward(self, x):
        # y = list(self.cv1(x).chunk(2, 1))
        y = [self.cv0(x), self.cv1(x)]
        y.extend(m(y[-1]) for m in self.m)
        return self.cv2(torch.cat(y, 1))


def transfer_weights(c2f, c2f_v2):
    c2f_v2.cv2 = c2f.cv2
    c2f_v2.m = c2f.m

    state_dict = c2f.state_dict()
    state_dict_v2 = c2f_v2.state_dict()

    # Transfer cv1 weights from C2f to cv0 and cv1 in C2f_v2
    old_weight = state_dict['cv1.conv.weight']
    half_channels = old_weight.shape[0] // 2
    state_dict_v2['cv0.conv.weight'] = old_weight[:half_channels]
    state_dict_v2['cv1.conv.weight'] = old_weight[half_channels:]

    # Transfer cv1 batchnorm weights and buffers from C2f to cv0 and cv1 in C2f_v2
    for bn_key in ['weight', 'bias', 'running_mean', 'running_var']:
        old_bn = state_dict[f'cv1.bn.{bn_key}']
        state_dict_v2[f'cv0.bn.{bn_key}'] = old_bn[:half_channels]
        state_dict_v2[f'cv1.bn.{bn_key}'] = old_bn[half_channels:]

    # Transfer remaining weights and buffers
    for key in state_dict:
        if not key.startswith('cv1.'):
            state_dict_v2[key] = state_dict[key]

    # Transfer all non-method attributes
    for attr_name in dir(c2f):
        attr_value = getattr(c2f, attr_name)
        if not callable(attr_value) and '_' not in attr_name:
            setattr(c2f_v2, attr_name, attr_value)

    c2f_v2.load_state_dict(state_dict_v2)


def replace_c2f_with_c2f_v2(module):
    for name, child_module in module.named_children():
        if isinstance(child_module, C2f):
            # Replace C2f with C2f_v2 while preserving its parameters
            shortcut = infer_shortcut(child_module.m[0])
            c2f_v2 = C2f_v2(child_module.cv1.conv.in_channels, child_module.cv2.conv.out_channels,
                            n=len(child_module.m), shortcut=shortcut,
                            g=child_module.m[0].cv2.conv.groups,
                            e=child_module.c / child_module.cv2.conv.out_channels)
            transfer_weights(child_module, c2f_v2)
            setattr(module, name, c2f_v2)
        else:
            replace_c2f_with_c2f_v2(child_module)


def save_model_v2(self: BaseTrainer):
    """
    Disabled half precision saving. originated from ultralytics/yolo/engine/trainer.py
    """
    ckpt = {
        'epoch': self.epoch,
        'best_fitness': self.best_fitness,
        'model': deepcopy(de_parallel(self.model)),
        'ema': deepcopy(self.ema.ema),
        'updates': self.ema.updates,
        'optimizer': self.optimizer.state_dict(),
        'train_args': vars(self.args),  # save as dict
        'date': datetime.now().isoformat(),
        'version': __version__}

    # Save last, best and delete
    torch.save(ckpt, self.last)
    if self.best_fitness == self.fitness:
        torch.save(ckpt, self.best)
    if (self.epoch > 0) and (self.save_period > 0) and (self.epoch % self.save_period == 0):
        torch.save(ckpt, self.wdir / f'epoch{self.epoch}.pt')
    del ckpt

def final_eval_v2(self: BaseTrainer):
    """
    originated from ultralytics/yolo/engine/trainer.py
    """
    for f in self.last, self.best:
        if f.exists():
            strip_optimizer_v2(f)  # strip optimizers
            if f is self.best:
                LOGGER.info(f'\nValidating {f}...')
                self.metrics = self.validator(model=f)
                self.metrics.pop('fitness', None)
                self.run_callbacks('on_fit_epoch_end')

def strip_optimizer_v2(f: Union[str, Path] = 'best.pt', s: str = '') -> None:
    """
    Disabled half precision saving. originated from ultralytics/yolo/utils/torch_utils.py
    """
    x = torch.load(f, map_location=torch.device('cpu'))
    args = {**DEFAULT_CFG_DICT, **x['train_args']}  # combine model args with default args, preferring model args
    if x.get('ema'):
        x['model'] = x['ema']  # replace model with ema
    for k in 'optimizer', 'ema', 'updates':  # keys
        x[k] = None
    for p in x['model'].parameters():
        p.requires_grad = False
    x['train_args'] = {k: v for k, v in args.items() if k in DEFAULT_CFG_KEYS}  # strip non-default keys
    # x['model'].args = x['train_args']
    torch.save(x, s or f)
    mb = os.path.getsize(s or f) / 1E6  # filesize
    LOGGER.info(f"Optimizer stripped from {f},{f' saved as {s},' if s else ''} {mb:.1f}MB")


def train_v2(self: YOLO, pruning=False, **kwargs):
    """
    Disabled loading new model when pruning flag is set. originated from ultralytics/yolo/engine/model.py
    """

    self._check_is_pytorch_model()
    if self.session:  # Ultralytics HUB session
        if any(kwargs):
            LOGGER.warning('WARNING ⚠️ using HUB training arguments, ignoring local training arguments.')
        kwargs = self.session.train_args
    overrides = self.overrides.copy()
    overrides.update(kwargs)
    if kwargs.get('cfg'):
        LOGGER.info(f"cfg file passed. Overriding default params with {kwargs['cfg']}.")
        overrides = yaml_load(check_yaml(kwargs['cfg']))
    overrides['mode'] = 'train'
    if not overrides.get('data'):
        raise AttributeError("Dataset required but missing, i.e. pass 'data=coco128.yaml'")
    if overrides.get('resume'):
        overrides['resume'] = self.ckpt_path

    self.task = overrides.get('task') or self.task
    self.trainer = TASK_MAP[self.task][1](overrides=overrides, _callbacks=self.callbacks)

    if not pruning:
        if not overrides.get('resume'):  # manually set model only if not resuming
            self.trainer.model = self.trainer.get_model(weights=self.model if self.ckpt else None, cfg=self.model.yaml)
            self.model = self.trainer.model

    else:
        # pruning mode
        self.trainer.pruning = True
        self.trainer.model = self.model

        # replace some functions to disable half precision saving
        self.trainer.save_model = save_model_v2.__get__(self.trainer)
        self.trainer.final_eval = final_eval_v2.__get__(self.trainer)

    self.trainer.hub_session = self.session  # attach optional HUB session
    self.trainer.train()
    # Update model and cfg after training
    if RANK in (-1, 0):
        self.model, _ = attempt_load_one_weight(str(self.trainer.best))
        self.overrides = self.model.args
        self.metrics = getattr(self.trainer.validator, 'metrics', None)

In [ ]:
def prune(args):
    # load trained yolov8 model
    model = YOLO(args.model)
    model.__setattr__("train_v2", train_v2.__get__(model))
    pruning_cfg = yaml_load(check_yaml(args.cfg))
    batch_size = pruning_cfg['batch']
    
    pruning_cfg['data'] = "coco128.yaml"
    pruning_cfg['epochs'] = 10
    
    model.model.train()
    replace_c2f_with_c2f_v2(model.model)
    initialize_weights(model.model)
    
    validation_model = deepcopy(model)
    metric = validation_model.val(**pruning_cfg)
    init_map = metric.box.map
    example_inputs = torch.randn(1, 3, pruning_cfg["imgsz"], pruning_cfg["imgsz"]).to(model.device)
    
    base_macs, base_nparams = tp.utils.count_ops_and_params(model.model, example_inputs)
    print(f"Before Pruning: MACs={base_macs / 1e9: .5f} G, #Params={base_nparams / 1e6: .5f} M, mAP={init_map: .5f}")
    
    for name, param in model.model.named_parameters():
        param.requires_grad = True
        
    model.train_v2(pruning=True, **pruning_cfg)

    pruning_cfg['epochs'] = 10
    
    macs_list, nparams_list, map_list, pruned_map_list = [], [], [], []
    base_macs, base_nparams = tp.utils.count_ops_and_params(model.model, example_inputs)
    
    pruning_cfg['name'] = f"baseline_val"
    pruning_cfg['batch'] = 1
    
    validation_model.model.model = deepcopy(model.model.model)
    metric = validation_model.val(**pruning_cfg)
    init_map = metric.box.map
    macs_list.append(base_macs)
    nparams_list.append(100)
    map_list.append(init_map)
    pruned_map_list.append(init_map)
    print(f"Before Pruning: MACs={base_macs / 1e9: .5f} G, #Params={base_nparams / 1e6: .5f} M, mAP={init_map: .5f}")
    
    for name, param in model.model.named_parameters():
        param.requires_grad = True
    
        ignored_layers = []
        unwrapped_parameters = []
        for m in model.model.modules():
            if isinstance(m, (Detect,)):
                ignored_layers.append(m)
    
    pruner = Pruner(model.model, 'local', large_final, ignored_layers=ignored_layers)
    print(model.model.model[0].conv)

    for i in range(args.iterative_steps):

        pruning_ratio = args.sched(args.target_prune_rate*100, i/args.iterative_steps)

        pruner.prune_model(pruning_ratio[0])
        print(pruning_ratio[0])

        print('After Pruning')
        print('Model', model.model.model[0].conv)
        print('Pruner', pruner.model.model[0].conv)

        pruning_cfg['name'] = f"step_{i}_pre_val"
        pruning_cfg['batch'] = 1
        validation_model.model.model = deepcopy(pruner.model.model)
        metric = validation_model.val(**pruning_cfg)
        pruned_map = metric.box.map
        pruned_macs, pruned_nparams = tp.utils.count_ops_and_params(pruner.model.to(default_device()), example_inputs.to(default_device()))
        
        print('After post-pruning Validation')
        print('Model', model.model.model[0].conv)
        print('Pruner', pruner.model.model[0].conv)
        
        
        current_speed_up = float(macs_list[0]) / pruned_macs
        print(f"After pruning iter {i + 1}: MACs={pruned_macs / 1e9} G, #Params={pruned_nparams / 1e6} M, "
              f"mAP={pruned_map}, speed up={current_speed_up}")

        
        # fine-tuning
        for name, param in model.model.named_parameters():
            param.requires_grad = True
        pruning_cfg['name'] = f"step_{i}_finetune"
        pruning_cfg['batch'] = batch_size  # restore batch size
        model.model = pruner.model
        model.train_v2(pruning=True, **pruning_cfg)

        print('After fine-tuning')
        print('Model', model.model.model[0].conv)
        print('Pruner', pruner.model.model[0].conv)
        
        
        # post fine-tuning validation
        pruning_cfg['name'] = f"step_{i}_post_val"
        pruning_cfg['batch'] = 1
        validation_model = YOLO(model.trainer.best)
        validation_model.model = deepcopy(model.model)
        metric = validation_model.val(**pruning_cfg)
        current_map = metric.box.map
        print(f"After fine tuning mAP={current_map}")

        print('After post fine-tuning validation')
        print('Model', model.model.model[0].conv)
        print('Pruner', pruner.model.model[0].conv)
    

        macs_list.append(pruned_macs)
        nparams_list.append(pruned_nparams / base_nparams * 100)
        pruned_map_list.append(pruned_map)
        map_list.append(current_map)

        if init_map - current_map > args.max_map_drop:
            print("Pruning early stop")
            break


    model.export(format='onnx')

## Training

In [ ]:
class Args(argparse.Namespace):
  model = 'yolov8l.pt'
  cfg = 'default.yaml'
  iterative_steps = 15
  target_prune_rate = 0.15
  max_map_drop = 0.2
  sched = Schedule(partial(sched_onecycle,  α=10, β=4))

args=Args()
prune(args)

Ultralytics YOLOv8.0.132 🚀 Python-3.9.0 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)
YOLOv8l summary (fused): 285 layers, 43668288 parameters, 0 gradients, 165.2 GFLOPs
val: Scanning /home/HubensN/fasterai/nbs/datasets/coco128/labels/train2017.cache... 126 images, 2 backgro
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████
                   all        128        929      0.731      0.768      0.828      0.659
                person        128        254      0.848      0.776      0.856      0.675
               bicycle        128          6      0.714      0.333      0.668      0.507
                   car        128         46      0.921      0.504      0.645      0.355
            motorcycle        128          5      0.825          1      0.995      0.874
              airplane        128          6      0.905          1      0.995       0.98
                   bus        128          7      0.751      0.714      0.8

Before Pruning: MACs= 82.72641 G, #Params= 43.69152 M, mAP= 0.65869


AMP: checks passed ✅
train: Scanning /home/HubensN/fasterai/nbs/datasets/coco128/labels/train2017.cache... 126 images, 2 backg
val: Scanning /home/HubensN/fasterai/nbs/datasets/coco128/labels/train2017.cache... 126 images, 2 backgro
Plotting labels to runs/detect/train46/labels.jpg... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 105 weight(decay=0.0), 112 weight(decay=0.0005), 111 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train46
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       1/10      10.7G     0.8537     0.7447      1.082        122        640: 100%|██████████| 8/8 [00:0
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████
                   all        128        929      0.776      0.741      0.832      0.667

      Epoch    GPU_mem   box_loss   cls

Before Pruning: MACs= 82.72641 G, #Params= 43.69152 M, mAP= 0.75438
Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)


Ultralytics YOLOv8.0.132 🚀 Python-3.9.0 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)


...

train: Scanning /home/HubensN/fasterai/nbs/datasets/coco128/labels/train2017.cache... 126 images, 2 backg
val: Scanning /home/HubensN/fasterai/nbs/datasets/coco128/labels/train2017.cache... 126 images, 2 backgro
Plotting labels to runs/detect/step_14_finetune12/labels.jpg... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 105 weight(decay=0.0), 112 weight(decay=0.0005), 111 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/step_14_finetune12
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       1/10      9.94G     0.4922     0.3236     0.8733        122        640: 100%|██████████| 8/8 [00:0
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████
                   all        128        929      0.955      0.882      0.946      0.822

      Epoch    GPU_mem   box_loss   cl

After fine-tuning
Model Conv2d(3, 54, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
Pruner Conv2d(3, 54, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)


YOLOv8l summary (fused): 285 layers, 32416863 parameters, 0 gradients, 123.4 GFLOPs
val: Scanning /home/HubensN/fasterai/nbs/datasets/coco128/labels/train2017.cache... 126 images, 2 backgro
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████
                   all        128        929      0.947      0.899      0.943      0.819
                person        128        254      0.977      0.848      0.949      0.778
               bicycle        128          6      0.895      0.667      0.792      0.568
                   car        128         46      0.963      0.562      0.731      0.385
            motorcycle        128          5      0.945          1      0.995      0.916
              airplane        128          6      0.966          1      0.995      0.924
                   bus        128          7      0.952          1      0.995      0.811
                 train        128          3      0.881          1      0.995    

After fine tuning mAP=0.8192818206570706
After post fine-tuning validation
Model Conv2d(3, 54, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
Pruner Conv2d(3, 54, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)


YOLOv8l summary (fused): 285 layers, 32416863 parameters, 0 gradients, 123.4 GFLOPs

PyTorch: starting from runs/detect/step_14_finetune12/weights/best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (124.2 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success ✅ 2.0s, saved as runs/detect/step_14_finetune12/weights/best.onnx (123.9 MB)

Export complete (2.8s)
Results saved to /home/HubensN/fasterai/nbs/runs/detect/step_14_finetune12/weights
Predict:         yolo predict task=detect model=runs/detect/step_14_finetune12/weights/best.onnx imgsz=640 
Validate:        yolo val task=detect model=runs/detect/step_14_finetune12/weights/best.onnx imgsz=640 data=/home/HubensN/miniconda3/envs/fasterai/lib/python3.9/site-packages/ultralytics/datasets/coco128.yaml 
Visualize:       https://netron.app


## Post-Training Checks

In [ ]:
model = YOLO('yolov8l.pt')

In [ ]:
example_inputs = torch.randn(1, 3, pruning_cfg["imgsz"], pruning_cfg["imgsz"]).to(model.device)
base_macs, base_nparams = tp.utils.count_ops_and_params(model.model, example_inputs); base_macs, base_nparams

(82726406400.0, 43691520)

In [ ]:
results = model.val(
                data='coco128.yaml',
                batch=1,
                imgsz=640,
                verbose=False,
            )

Ultralytics YOLOv8.0.132 🚀 Python-3.9.0 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)
YOLOv8l summary (fused): 268 layers, 43668288 parameters, 0 gradients, 165.2 GFLOPs
val: Scanning /home/HubensN/fasterai/nbs/datasets/coco128
                 Class     Images  Instances      Box(P  
                   all        128        929      0.833      0.694       0.83      0.659
Speed: 0.2ms preprocess, 22.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/val33


In [ ]:
results

ultralytics.yolo.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 79])
box: ultralytics.yolo.utils.metrics.Metric object
confusion_matrix: <ultralytics.yolo.utils.metrics.ConfusionMatrix object>
fitness: 0.675906972569436
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.67313,     0.52749,     0.35361,     0.82095,     0.96459,     0.73294,     0.96262,     0.33383,     0.59455,     0.27281,     0.65884,      0.8955,     0.65884,      0.5898,     0.69265,      0.9071,     0.90664,     0.79925,     0.65884,     0.65884,     0.85369,      0.8955,     0.97279,     0.81737,
           0.49689,     0.69167,     0.37624,     0.66

In [ ]:
model = YOLO('/home/HubensN/fasterai/nbs/runs/detect/step_14_finetune4/weights/best.pt')

In [ ]:
base_macs, base_nparams = tp.utils.count_ops_and_params(model.model, example_inputs); base_macs, base_nparams

(57692198400.0, 30077028)

In [ ]:
results = model.val(
                data='coco128.yaml',
                batch=1,
                imgsz=640,
                verbose=False,
            )

Ultralytics YOLOv8.0.132 🚀 Python-3.9.0 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)
YOLOv8l summary (fused): 285 layers, 30057792 parameters, 0 gradients, 115.1 GFLOPs
val: Scanning /home/HubensN/fasterai/nbs/datasets/coco128/labels/tra
                 Class     Images  Instances      Box(P          R  
                   all        128        929      0.917      0.907      0.945      0.809
Speed: 0.2ms preprocess, 24.4ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/val35


In [ ]:
results

ultralytics.yolo.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 79])
box: ultralytics.yolo.utils.metrics.Metric object
confusion_matrix: <ultralytics.yolo.utils.metrics.ConfusionMatrix object>
fitness: 0.8221835652536718
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.75668,     0.47387,      0.3594,     0.91994,     0.94661,     0.90211,     0.94289,     0.68531,     0.68927,     0.34436,     0.80851,      0.8955,     0.80851,     0.86091,     0.84563,     0.96863,     0.89911,      0.9501,     0.80851,     0.80851,     0.88959,       0.995,       0.995,     0.93382,
            0.8273,     0.84511,      0.6686,     0.7

In [ ]:
model.export(format = 'onnx', half = True)

Ultralytics YOLOv8.0.132 🚀 Python-3.9.0 torch-2.2.1 CPU
WARNING ⚠️ half=True only compatible with GPU export, i.e. use device=0
YOLOv8l summary (fused): 268 layers, 43668288 parameters, 0 gradients, 165.2 GFLOPs

PyTorch: starting from yolov8l.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (83.7 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success ✅ 2.8s, saved as yolov8l.onnx (166.8 MB)

Export complete (4.0s)
Results saved to /home/HubensN/fasterai/nbs
Predict:         yolo predict task=detect model=yolov8l.onnx imgsz=640 
Validate:        yolo val task=detect model=yolov8l.onnx imgsz=640 data=coco.yaml 
Visualize:       https://netron.app


'yolov8l.onnx'

In [ ]:
model = YOLO('/home/HubensN/fasterai/nbs/runs/detect/step_14_finetune4/weights/best.onnx')

In [ ]:
results = model.val(
                data='coco128.yaml',
                batch=1,
                imgsz=640,
                verbose=False,
            )

Ultralytics YOLOv8.0.132 🚀 Python-3.9.0 torch-2.2.1 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)
Loading yolov8l.onnx for ONNX Runtime inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models
val: Scanning /home/HubensN/fasterai/nbs/datasets/coco128/labels/tra
                 Class     Images  Instances      Box(P          R  
                   all        128        929      0.731      0.767      0.828      0.658
Speed: 0.2ms preprocess, 13.1ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/val39
